# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra code. 

## <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.
## Columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project_2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity_project_2')
except Exception as e:
    print(e)

# Table songs_by_session

## For table songs_by_session, the sessionId column was used as a partition key because the queries will filter by this column. itemInSession was used as clustering columns to help make up a unique key.

In [17]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session "
query = query + """
(sessionId int, 
 itemInSession int, 
 artist text, 
 length float, 
 song text, 
 PRIMARY KEY (sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Table songs_by_user

## For table songs_by_user, the userid column and the sessionId column were used as the partition key because the queries will filter by this columns. itemInSession was used as clustering columns to help make up a unique key.

In [ ]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user "
query = query + """
(userId int, 
 sessionId int, 
 itemInSession int, 
 artist text, 
 firstName text, 
 lastName text, 
 song text, 
 PRIMARY KEY ((userid, sessionId), itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Table username_by_song

## For table username_by_song, the song column was used as a partition key because the queries will filter by this column. userid was used as clustering columns to help make up a unique key.

In [ ]:
query = "CREATE TABLE IF NOT EXISTS username_by_song "
query = query + """
(song text, 
 userId int,
 firstName text, 
 lastName text,
 PRIMARY KEY (song, userid))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:


        query = """
        INSERT INTO songs_by_session (sessionId, itemInSession, artist, length, song)
        """
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        except Exception as e:
            print(e)
        
        
        query = """
        INSERT INTO songs_by_user (userId, sessionId, itemInSession, artist, firstName, lastName, song)
        """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        except Exception as e:
            print(e)
            
        query = """
        INSERT INTO username_by_song (song, userId, firstName, lastName)
        """
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)
        


#### SELECT to verify that the data have been inserted into each table

In [19]:
query = "select artist from songs_by_session WHERE sessionId = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist)

Pixies
The Roots / Jack Davey
Mike And The Mechanics
Faithless


## Query 1
Get me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [20]:
query = "select artist, song, length from songs_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2
Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [24]:
query = "select artist, song, firstname, lastname from songs_by_user WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3
Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [25]:
query = "select firstname, lastname from username_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [16]:

query = "DROP TABLE IF EXISTS songs_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS songs_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS username_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()